Copyright (c) MONAI Consortium  
Licensed under the Apache License, Version 2.0 (the "License");  
you may not use this file except in compliance with the License.  
You may obtain a copy of the License at  
    http://www.apache.org/licenses/LICENSE-2.0  
Unless required by applicable law or agreed to in writing, software  
distributed under the License is distributed on an "AS IS" BASIS,  
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  
See the License for the specific language governing permissions and  
limitations under the License. 

# MONAI Label Radiology App - OHIF Spleen Segmentation Example

***The OHIF + MONAI Label integration with spleen segmentation***

In this notebook, we show end-to-end setup of the web-based OHIF viewer and MONAI Label plugin. The spleen segmentation is demonstrated.

<p align = "center"><img src="./figures/monailabel_radiology_spleen_segmentation_OHIF/ohif0.png" alt="drawing" width="900"/></p>


## 1. Prepare MONAI Label

### Prerequisites
- **Install MONAI Label** weekly preview release: 


In [ ]:
!pip install monailabel-weekly

### 1.1 Download Radiology app
Choose "radiology" as the app

In [ ]:
!monailabel apps --download --name radiology --output apps

## 2. Setup Orthanc DICOM Server

OHIF works well with the web-based Orthanc DICOM Server. Orthanc provides a simple, and powerful standalone DICOM server. It is designed to improve the DICOM flows in hospitals and to support research about the automated analysis of medical images. For more details on Orthanc, please refer to the [Orthanc](https://www.orthanc-server.com/index.php) page. 

The official [download page](https://www.orthanc-server.com/download.php) provides detailed steps for installing Orthanc.

```bash
# Install orthanc and dicomweb plugin
sudo apt-get install orthanc orthanc-dicomweb -y

# stop the existing Orthanc instance if there is one
sudo service orthanc stop

# setup and upgrade Orthanc libraries
sudo wget https://lsb.orthanc-server.com/orthanc/1.9.7/Orthanc --output-document /usr/sbin/Orthanc
sudo rm -f /usr/share/orthanc/plugins/*.so

sudo wget https://lsb.orthanc-server.com/orthanc/1.9.7/libServeFolders.so --output-document /usr/share/orthanc/plugins/libServeFolders.so
sudo wget https://lsb.orthanc-server.com/orthanc/1.9.7/libModalityWorklists.so --output-document /usr/share/orthanc/plugins/libModalityWorklists.so
sudo wget https://lsb.orthanc-server.com/plugin-dicom-web/1.6/libOrthancDicomWeb.so --output-document /usr/share/orthanc/plugins/libOrthancDicomWeb.so

# start
sudo service orthanc restart
```

After the above steps, the Orthanc will up by default at

**http://127.0.0.1:8042**

The index page of Orthanc DICOM Web page:

<p align = "center"><img src="./figures/monailabel_radiology_spleen_segmentation_OHIF/ohif1.png" alt="drawing" width="900"/></p>



## 3. Prepare Dicom Sample data

### 3.1 Download Spleen CT Data
Download MSD Spleen dataset as the sample dataset using monailabel API. The task is the volumetric (3D) segmentation of the spleen from CT image. The segmentation of the spleen is formulated as the voxel-wise 2-class classification. Each voxel is predicted as either foreground (spleen) or background. The dataset is from the 2018 MICCAI challenge.

In [ ]:
!monailabel datasets --download --name Task09_Spleen --output datasets

### 3.2 Convert to Dicom Format

To use the DICOM server, the NIFTI format data needs to be converted to DICOM format, plastimatch is an open tool to generate simple DICOM data from NIFTI file. The following commands show how to convert a sample DICOM data.

```bash
# Install `plastimatch` NIFTI to DICOM converter
sudo apt-get install plastimatch -y

# convert one of the NIFTI images to DICOM: name: <patient1>, output folder: <dicom_output>
plastimatch convert --patient-id patient1 --input datasets/Task09_Spleen/imagesTs/spleen_1.nii.gz --output-dicom dicom_output
```

### 3.3 Upload to Orthanc

- Click `Upload` on the top right corner of Orthanc index page. 

<p align = "center"><img src="./figures/monailabel_radiology_spleen_segmentation_OHIF/ohif2.png" alt="drawing" width="900"/></p>

- Click `Select files to upload` and select **all** DICOM files of each subject.
- Click `Start the upload` to upload to Orthanc server.
- Users can check the uploaded DICOM file for each subject by clicking `All `studies` on the index page.

<p align = "center"><img src="./figures/monailabel_radiology_spleen_segmentation_OHIF/ohif3.png" alt="drawing" width="900"/></p>

## 4. Start MONAI Label Server

The command will start MONAI Label server with all available models. If the user wants to start with a specific model, set the model name in the **--conf models** argument.

Such as

```bash
monailabel start_server --app <path to pathology app> --studies <Orthanc server URL> --conf models <segmentation_spleen>
```
In the command, ```--app``` specifies the pathology app path, ```--studies``` specifies the datastore URL.

Note: by default, the studies argument is specified as: 

**http://127.0.0.1:8042/dicom-web**

In [ ]:
!monailabel start_server --app apps/radiology --studies http://127.0.0.1:8042/dicom-web --conf models segmentation_spleen

## 5. Start OHIF Viewer

After the monailabel server is up. Users can access OHIF by:

**http://127.0.0.1:8000/ohif/**

Note: by default is monailabel server is up with port 8000, if port 8000 is occupied by another application, users can specify the port number by `--port <port #>` in monailabel start_server command.
Open the OHIF viewer, users can see the subject list as below:

<p align = "center"><img src="./figures/monailabel_radiology_spleen_segmentation_OHIF/ohif4.png" alt="drawing" width="900"/></p>

## 6. Auto Segmentation with OHIF

Below is a step-by-step tutorial on using OHIF for auto-spleen segmentation. Pre-trained model is provided by the MONAI Label.

### 6.1 MONAI Label Plugin

- MONAI Label plugin is already build-in. Click the subject to annotate, in the labeling page, click `MONAI `Label` button in the top-right corner.

<p align = "center"><img src="./figures/monailabel_radiology_spleen_segmentation_OHIF/ohif5.png" alt="drawing" width="900"/></p>

### 6.2 Run Auto Segmentation

- MONAI Label has loaded the pre-trained weights, click **Run** button in the right MONAI Label plugin panel. 

- Note: the pre-trained models will be downloaded when starting MONAI Label server in the "**model**" folder. For instance, in this use case, the downloaded pre-trained model is saved at "**apps/radiology/model/pretrained_segmentation_spleen.pt**".

Users can monitor the logs in the MONAI Label server terminal.

The auto segmentation results can show up as below:

<p align = "center"><img src="./figures/monailabel_radiology_spleen_segmentation_OHIF/ohif6.png" alt="drawing" width="900"/></p>

### 6.3 Edit Annotations and Submit Label

- Users can edit the inference labels manually using the OHIF segmentation editor, the tools for adjusting annotation. Once done annotation, click **Submit **Label** on the right plugin panel to save the annotation.

- Note: the final annotations will be saved to "**labels/final**" folder in the study dataset, for example, in this use case, the ground truth label will be saved to "**datasets/Task09_Spleen/imagesTr/labels/final**"

<p align = "center"><img src="./figures/monailabel_radiology_spleen_segmentation_OHIF/ohif7.png" alt="drawing" width="900"/></p>

## 7. Training and Active Learning

Active learning and interactive fine-tuning models have highlighted features in MONAI Label, users can train their models anytime when new annotated labels are saved. Click **Update Model** button, MONAI Label server will fetch the saved final ground truth label and fine-tune the prior model. 

Advanced Feature: users can set training parameters in the `options` panel.

<p align = "center"><img src="./figures/monailabel_radiology_spleen_segmentation_OHIF/ohif8.png" alt="drawing" width="400"/></p>

Users can do auto-segmentation using the latest fine-tuned model. 

<p align = "center"><img src="./figures/monailabel_radiology_spleen_segmentation_OHIF/ohif9.png" alt="drawing" width="400"/></p>

The next image is selected, and the already trained images will be marked as labeled data, then won't be selected for the next labeling batch. Active learning strategies such as "first/random" will be used for selecting which unlabeled data to fetch. 

## 8. Conclusion

This tutorial demonstrates the basic usage of MONAI Label and OHIF. Use spleen segmentation and radiology app as examples, we provide instructions on how to set up DICOM web server: Orthanc, how to set up OHIF, how to start monailabel server, and basic use cases of auto segmentation, model training and active learning.